In [1]:
with open('news.txt', 'r') as f:
    text = f.read()
    news = text.split("\n\n")
    count = {'sport': 0, 'world': 0, "us": 0, "business": 0, "health": 0, "entertainment": 0, "sci_tech": 0}
    for news_item in news:
        lines = news_item.split("\n")
        # if len(lines) == 7:
        print(lines[6])
        file_to_write = open('data/' + lines[6] + '/' + str(count[lines[6]]) + '.txt', 'w+')
        count[lines[6]] = count[lines[6]] + 1
        file_to_write.write(news_item)  # python will convert \n to os.linesep
        file_to_write.close()


sport
business
sport
entertainment
us
entertainment
world
business
business
world
us
sport
entertainment
sport
world
health
world
world
us
entertainment
sport
us
us
sport
entertainment
world
business
sport
world
sci_tech
sport
entertainment
business
us
sport
world
world
sport
us
world
sci_tech
sport
health
entertainment
business
us
entertainment
entertainment
sport
sport
entertainment
us
entertainment
business
us
sport
sport
sport
entertainment
business
business
sport
entertainment
us
world
sport
sci_tech
sport
us
sport
business
sport
us
business
us
us
us
world
entertainment
business
world
business
sport
sport
sport
business
world
health
sport
sci_tech
world
entertainment
entertainment
sport
world
world
health
health
business
world
sport
sport
sport
sport
health
world
sport
sport
world
sport
world
world
health
world
world
sport
business
us
us
world
sport
health
business
sport
sport
world
world
us
sport
us
world
business
entertainment
us
entertainment
sci_tech
us
us
business
us
entertai

IndexError: list index out of range

In [2]:
import pandas
import glob

category_list = ["sport", "world", "us", "business", "health", "entertainment", "sci_tech"]
directory_list = ["data/sport/*.txt", "data/world/*.txt","data/us/*.txt","data/business/*.txt","data/health/*.txt","data/entertainment/*.txt","data/sci_tech/*.txt",]

text_files = list(map(lambda x: glob.glob(x), directory_list))
text_files = [item for sublist in text_files for item in sublist]

training_data = []


for t in text_files:
    f = open(t, 'r')
    f = f.read()
    t = f.split('\n')
    training_data.append({'data' : t[0] + ' ' + t[1], 'flag' : category_list.index(t[6])})
    
training_data[0]

{'data': 'justice department will talk with bcs executive director bill hancock justice department officials want to talk with bowl championship series executive director bill hancock as part of their inquiry into whether ...',
 'flag': 0}

In [3]:
training_data = pandas.DataFrame(training_data, columns=['data', 'flag'])
training_data.to_csv("train_data.csv", sep=',', encoding='utf-8')
print(training_data.data.shape)


(32602,)


In [4]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer


#GET VECTOR COUNT
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(training_data.data)

#SAVE WORD VECTOR
pickle.dump(count_vect.vocabulary_, open("count_vector.pkl","wb"))

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer

#TRANSFORM WORD VECTOR TO TF IDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#SAVE TF-IDF
pickle.dump(tfidf_transformer, open("tfidf.pkl","wb"))

In [6]:
# Multinomial Naive Bayes

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

#clf = MultinomialNB().fit(X_train_tfidf, training_data.flag)
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.25, random_state=42)
clf = MultinomialNB().fit(X_train, y_train)

#SAVE MODEL
pickle.dump(clf, open("nb_model.pkl", "wb"))

In [7]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

category_list = ["sport", "world", "us", "business", "health", "entertainment", "sci_tech"]

docs_new = "Science and technology"
docs_new = [docs_new]

#LOAD MODEL
loaded_vec = CountVectorizer(vocabulary=pickle.load(open("count_vector.pkl", "rb")))
loaded_tfidf = pickle.load(open("tfidf.pkl","rb"))
loaded_model = pickle.load(open("nb_model.pkl","rb"))

X_new_counts = loaded_vec.transform(docs_new)
X_new_tfidf = loaded_tfidf.transform(X_new_counts)
predicted = loaded_model.predict(X_new_tfidf)
print(type(X_new_tfidf))

print(category_list[predicted[0]])






<class 'scipy.sparse._csr.csr_matrix'>
sci_tech


In [8]:
predicted = loaded_model.predict(X_test)
result_bayes = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_bayes.to_csv('res_bayes.csv', sep = ',')

for predicted_item, result in zip(predicted, y_test):
    print(category_list[predicted_item], ' - ', category_list[result])
# Don't run this



entertainment  -  entertainment
world  -  world
sport  -  entertainment
business  -  business
sport  -  sport
us  -  us
business  -  business
business  -  business
sport  -  entertainment
business  -  sci_tech
sport  -  sport
sport  -  sport
world  -  world
world  -  world
sci_tech  -  sci_tech
world  -  health
sport  -  sci_tech
sport  -  sport
us  -  us
sport  -  sport
business  -  business
sport  -  sport
sport  -  sci_tech
us  -  us
world  -  us
entertainment  -  entertainment
entertainment  -  entertainment
world  -  world
world  -  business
sport  -  sport
sport  -  sport
business  -  sci_tech
sport  -  world
us  -  us
world  -  world
entertainment  -  entertainment
sport  -  sport
world  -  world
world  -  world
world  -  world
sport  -  sport
sci_tech  -  sci_tech
sport  -  sport
world  -  world
business  -  business
entertainment  -  entertainment
sport  -  sport
world  -  world
world  -  world
sport  -  us
sport  -  entertainment
sport  -  sport
business  -  health
us  -  us


In [9]:
from sklearn.metrics import confusion_matrix  

confusion_mat = confusion_matrix(y_test,predicted)
print(confusion_mat)

[[2021    3    6    4    0    0    0]
 [  22 1485   15   45    0    9    0]
 [ 111  158  816  112    0    6    4]
 [  47   49   41 1117    3   13   27]
 [  96   55   85   84  136    3    3]
 [ 249   45   26   27    0  506    6]
 [  69   63   53  294    2    5  230]]


In [10]:
from sklearn.neural_network import MLPClassifier

clf_neural = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)

X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.25, random_state=42)

clf_neural.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(15,), random_state=1,
              solver='lbfgs')

In [11]:
pickle.dump(clf_neural, open("softmax.pkl", "wb"))

In [12]:
predicted = clf_neural.predict(X_test)
print(X_test)
result_softmax = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_softmax.to_csv('res_softmax.csv', sep = ',')

for predicted_item, result in zip(predicted, y_test):
    print(category_list[predicted_item], ' - ', category_list[result])


  (0, 35974)	0.17622964820079523
  (0, 35892)	0.15962482877633535
  (0, 35151)	0.15433957919991442
  (0, 32614)	0.1097389956806214
  (0, 32455)	0.24827702460243187
  (0, 32454)	0.10747354871893752
  (0, 32258)	0.19825815536655533
  (0, 30650)	0.12837991477279892
  (0, 30097)	0.4613771520285778
  (0, 27151)	0.12173624928466129
  (0, 26768)	0.1887315033185514
  (0, 22467)	0.07109593076604605
  (0, 21319)	0.1328425485574192
  (0, 21245)	0.13745305417245
  (0, 19202)	0.12856822749912303
  (0, 18896)	0.10836225130567204
  (0, 17012)	0.059918354840258786
  (0, 16492)	0.21574316517005346
  (0, 13006)	0.22194607111253983
  (0, 10951)	0.10900144090515333
  (0, 10134)	0.14790357185081254
  (0, 8520)	0.14655552853848244
  (0, 4007)	0.16043857327810795
  (0, 3729)	0.21574316517005346
  (0, 2847)	0.4140013205366087
  :	:
  (8150, 32822)	0.039122787805542646
  (8150, 32454)	0.03106826238206182
  (8150, 32392)	0.15526110959323985
  (8150, 32018)	0.1587741695683553
  (8150, 29535)	0.20812359861017182


In [13]:
from sklearn import svm
clf_svm = svm.LinearSVC()
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.25, random_state=42)
clf_svm.fit(X_train_tfidf, training_data.flag)
pickle.dump(clf_svm, open("svm.pkl", "wb"))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [14]:
predicted = clf_svm.predict(X_test)
result_svm = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_svm.to_csv('res_svm.csv', sep = ',')
for predicted_item, result in zip(predicted, y_test):
    print(category_list[predicted_item], ' - ', category_list[result])

entertainment  -  entertainment
world  -  world
entertainment  -  entertainment
business  -  business
sport  -  sport
us  -  us
business  -  business
business  -  business
entertainment  -  entertainment
sci_tech  -  sci_tech
sport  -  sport
sport  -  sport
world  -  world
world  -  world
sci_tech  -  sci_tech
health  -  health
sci_tech  -  sci_tech
sport  -  sport
us  -  us
sport  -  sport
business  -  business
sport  -  sport
sci_tech  -  sci_tech
us  -  us
us  -  us
entertainment  -  entertainment
entertainment  -  entertainment
world  -  world
business  -  business
sport  -  sport
sport  -  sport
sci_tech  -  sci_tech
world  -  world
us  -  us
world  -  world
entertainment  -  entertainment
sport  -  sport
world  -  world
world  -  world
world  -  world
sport  -  sport
sci_tech  -  sci_tech
sport  -  sport
world  -  world
sci_tech  -  business
entertainment  -  entertainment
sport  -  sport
world  -  world
world  -  world
us  -  us
entertainment  -  entertainment
sport  -  sport
he